# Задача
Нейронная сеть: поэкспериментировать с другими простыми фигурами — эллипсом, квадратом, треугольником — повёрнутыми на разные углы (аналитическая геометрия в помощь).

Начнём с эллипса. Зададим его, скажем, полуосями и углом поворота между большой полуосью и осью OX.

In [3]:
import os
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input
from keras.utils import np_utils

In [8]:
def saturate(v):
    return min(1, max(0, v))

import matplotlib.pyplot as plt
def plot_from_model(filename, smooth=False):
    if(os.path.isfile(filename)):
        model = load_model(filename)
    else:
        print(f"No file found:{filename}")
        return
    %matplotlib inline
    plt.axis('equal')
    plt.rcParams["figure.figsize"] = [16,9]
    
    c = np.r_[-2:2:0.1]
    # https://stackoverflow.com/a/11144716/539470 =)
    XY = np.transpose([np.tile(c, len(c)), np.repeat(c, len(c))])
    Z = model.predict(XY)
    
    
    if smooth:
        for (x, y), z in zip(XY, Z):
            plt.scatter(x, y, color=[(1, 1-saturate(z[0]), 1-saturate(z[0]))])
    else:
        for (x, y), z in zip(XY, Z):
            plt.scatter(x, y, c='red' if z[0] >= 0.5 else 'green')
    plt.show() 

In [14]:
from math import pi,sin,cos
def rotate(A,angle=0):
    return (A[0]*cos(angle)+A[1]*sin(angle),A[1]*cos(angle)-A[0]*sin(angle))
    #Вероятно, это можно оптимизировать через numpy-евское умножение вектора координат на матрицу поворота,
    #просчитанную заранее
def inEllipse(A,a=1,b=1,angle=0):
    #a - большая, b-малая, angle - угол поворота в радианах
    B=rotate(A,angle)#Координаты в системе координат эллипса
    #В ней уравнение эллипса имеет простую форму:
    return 1 if (B[0]/a)**2+(B[1]/b)**2<=1 else 0

def teach_model(X,iters,filename,teacher):
    #teacher - обучающая функция, должна быть (x,y) |--> h(x,y) из [0,1]
    Y=[teacher(a) for a in X]
    l0 = Input(shape=(2,))
    l1 = Dense(6, activation='sigmoid', use_bias=True)(l0)
    l2 = Dense(1, activation='sigmoid', use_bias=False)(l1)

    model = Model(input=l0, output=l2)

    model.compile(
        loss='mean_squared_error',
        optimizer='adam',
        metrics=['accuracy']
    )
    model.fit(
            X, Y,
            epochs=10000,
            verbose=False
        )
    model.save(filename)
def angleToRads(phi):
    return phi/360*2*pi

Для начала попробуем обучать на простой решетке, для нескольких разных углов.

In [10]:
#making a grid

c = np.r_[-2:2:0.2]
#Не брать a больше 2, не меняя границы решетки
# https://stackoverflow.com/a/11144716/539470 =)
grid = np.transpose([np.tile(c, len(c)), np.repeat(c, len(c))])
print(len(c)**2)
grid[0]

400


array([-2., -2.])

In [15]:
import time
angles = [0,angleToRads(10),angleToRads(30),angleToRads(111.5)]
a=1.5
b=0.5
for angle in angles:
    sTime=time.time()
    filename = f"models/Ellipse_angle{round(angle,2}_a{a}_b{b}.h5"
    if(not os.path.isfile(filename)):
        teach_model(grid,10000,filename,lambda A:inEllipse(A,a,b,angle))
    plot_from_model(filename,smooth=True)
    fTime=time.time()
    print(f"Plot for angle={round(angle,2)}")

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


KeyboardInterrupt: 

In [ ]:
from math import pi, sin,cos
#making two bounding circles close to the original
bounding = np.zeros((200,2))
for i in range(100):
    bounding[i][0]=0.95*sin(i/100*2*pi)
    bounding[i][1]=0.95*cos(i/100*2*pi)
    bounding[i+100][0]=1.05*sin(i/100*2*pi)
    bounding[i+100][1]=1.05*cos(i/100*2*pi)
print(len(bounding))
print(bounding[0])